In [2]:
import numpy as np
# import utils
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
class DenseLayer:
    
    def __init__(self, n_nodes,  activation = 'sigmoid', input_shape = None, use_bias = True):
        
        self.n_nodes = n_nodes
        self.activation = activation
        self.input_shape = input_shape
        self.use_bias = use_bias


In [237]:
class NeuralNetwork:
    
    def __init__(self, learning_rate = 0.0005, layers = None, weights=  None, biases = None, activations = None):
        
        self.alpha = learning_rate
        
        self.use_bias = True
        
        if layers is not None:
            self.layers = layers
        else:
            self.layers = []
            
        if weights is not None:
            self.weights = weights
        else:
            self.weights = []
            
        if biases is not None:
            self.biases = biases
        else:
            self.biases = []
            
        if activations is not None:
            self.activations = activations
        else:
            self.activations = []
            
        self.test_accuracy = 0
        self.train_accuracy = 0
        self.n_layers = 0
        self.nodes_per_layer = []
        self.layer_biases = []
    
    def add (self, layer):
        
        if (layer.input_shape is not None):
            
            (n_nodes, _) = layer.input_shape
#             input_layer = np.random.randn(x, y)
            self.layers.append(None)
            self.nodes_per_layer.append(n_nodes)
            self.activations.append(layer.activation)
            
            if (layer.use_bias):
                self.layer_biases.append(0)
            else:
                self.layer_biases.append(None)
                
            self.n_layers += 1
            return
        
        self.weights.append(np.random.randn(self.nodes_per_layer[-1], layer.n_nodes))
#         new_layer = np.random.randn(layer.n_nodes,self.layers[-1].shape[1])
        self.layers.append(None)
        self.nodes_per_layer.append(layer.n_nodes)
        self.activations.append(layer.activation)
        self.n_layers += 1
        
        if self.use_bias:
            self.biases.append(np.random.randn(1, layer.n_nodes))
        else:
            self.biases.append(None)
            
        if (layer.use_bias):
            self.layer_biases.append(0)
        else:
            self.layer_biases.append(None)
                
        self.use_bias = layer.use_bias
        
    def apply_activation (self, layer_num):
        
        if self.activations[layer_num] is 'identity':
            return self.layers[layer_num]
        
        elif self.activations[layer_num] is 'sigmoid':
            return utils.sigmoid (self.layers[layer_num])
        
        elif self.activations[layer_num] is 'relu':
            return utils.relu (self.layers[layer_num])
        
        elif self.activations[layer_num] is 'leaky_relu':
            return utils.leaky_relu (self.layers[layer_num])
        
        elif self.activations[layer_num] is 'tanh':
            return utils.tanh (self.layers[layer_num])
        
        else:
            raise Exception('Invalid activation for layer number {}'.format(layer_num))
        
    def fit (self, input_vector, label):
        
        input_vector = np.reshape (input_vector, (input_vector.shape[0], -1))
        
        # Forward propogation
        
        self.layers[0] = input_vector
        
        for (layer_num, layer) in enumerate(self.layers):
            
            if self.layer_biases[layer_num] is not None:
                self.layer_biases[layer_num] = np.random.randn(1, input_vector.shape[1])
            
            if layer_num is 0:
                continue
                
            self.layers[layer_num] = np.matmul(self.apply_activation(layer_num - 1).T, self.weights[layer_num - 1]).T # + self.biases[layer_num - 1])
            
            if self.layer_biases[layer_num - 1] is not None:
                self.layers[layer_num] += np.matmul(self.layer_biases[layer_num - 1].T, self.biases[layer_num - 1]).T
            
        self.layers[self.n_layers - 1][self.layers[self.n_layers - 1] > 15] = 15
        self.layers[self.n_layers - 1][self.layers[self.n_layers - 1] < -15] = -15
        
        prediction = self.apply_activation(len(self.layers)-1)
        error = np.sum (-(label * np.log(prediction) +  (1 - label) * np.log(1 - prediction)), axis = 1)
        
        self.train_accuracy += np.sum(((prediction>=0.5) - label)**2, axis = 1)
        self.train_accuracy = self.train_accuracy / prediction.shape[1]
        
        
        # Backward Propogation
    
        delta = prediction - label

        for (l, layer) in enumerate(reversed(self.layers)):

            layer_num = len(self.layers) - 1 - l

            if layer_num is 0:
                break

            dW = np.matmul(self.layers[layer_num - 1], delta.T)

            if self.layer_biases[layer_num - 1] is not None:
                    dB = np.matmul(self.layer_biases[layer_num - 1], delta.T)

            delta = utils.calc_der(self.activations[layer_num - 1], self.layers[layer_num - 1]) * (np.matmul(self.weights[layer_num - 1], delta))

            self.weights[layer_num - 1] -= self.alpha * dW

            if self.biases[layer_num - 1] is not None:
                self.biases[layer_num - 1] -= self.alpha * dB
                
    def predict (self, input_vector, label):
        
            input_vector = np.reshape (input_vector, (input_vector.shape[0], -1))

            self.layers[0] = input_vector

            for (layer_num, layer) in enumerate(self.layers):

                if layer_num is 0:
                    continue

                self.layers[layer_num] = (np.matmul(self.apply_activation(layer_num - 1).T, self.weights[layer_num - 1]).T) # + self.biases[layer_num - 1])
                
                if self.layer_biases[layer_num - 1] is not None:
                    self.layers[layer_num] += np.matmul(self.layer_biases[layer_num - 1].T, self.biases[layer_num - 1]).T
            
            self.layers[self.n_layers - 1][self.layers[self.n_layers - 1] > 15] = 15
            self.layers[self.n_layers - 1][self.layers[self.n_layers - 1] < -15] = -15

            prediction = self.apply_activation(len(self.layers)-1)
            print("pred", prediction)
            
            self.test_accuracy += np.sum(((prediction>=0.5) - label)**2, axis = 1)
            self.test_accuracy = self.test_accuracy / prediction.shape[1]
            

In [3]:
data = pd.read_csv('../datasets/housepricedata.csv', header = None)

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,10
0,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
1,8450,7,5,856,2,1,3,8,0,548,1
2,9600,6,8,1262,2,0,3,6,1,460,1
3,11250,7,5,920,2,1,3,6,1,608,1
4,9550,7,5,756,1,0,3,7,1,642,0
...,...,...,...,...,...,...,...,...,...,...,...
1456,7917,6,5,953,2,1,3,7,1,460,1
1457,13175,6,6,1542,2,0,3,7,2,500,1
1458,9042,7,9,1152,2,0,4,9,2,252,1
1459,9717,5,6,1078,1,0,2,5,0,240,0


In [4]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
1,8450,7,5,856,2,1,3,8,0,548,1
2,9600,6,8,1262,2,0,3,6,1,460,1
3,11250,7,5,920,2,1,3,6,1,608,1
4,9550,7,5,756,1,0,3,7,1,642,0


In [11]:
data.sample(frac = 1)

,0,1,2,3,4,5,6,7,8,9,10
55,7134,5,5,384,1,0,3,6,1,572,0
914,6270,5,6,1001,2,0,4,8,0,871,0
893,8414,6,8,1059,1,0,3,6,0,264,0
1353,6000,6,9,698,1,0,2,4,0,624,0
624,2117,6,5,756,2,1,2,4,1,440,1
...,...,...,...,...,...,...,...,...,...,...,...
1163,8724,5,5,894,1,0,3,5,1,450,0
743,8450,7,5,1349,2,0,3,6,0,539,1
211,5604,5,6,864,1,0,2,5,0,0,0
1397,57200,5,5,747,1,0,3,7,2,572,0


In [12]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
1,8450,7,5,856,2,1,3,8,0,548,1
2,9600,6,8,1262,2,0,3,6,1,460,1
3,11250,7,5,920,2,1,3,6,1,608,1
4,9550,7,5,756,1,0,3,7,1,642,0


In [78]:
X = data.iloc[1:,:-1].astype(int)
Y = data.iloc[1:,-1].astype(int)

In [79]:
X = np.array(X)

In [80]:
X_min = np.min(X, axis = 0)
X_max = np.max(X, axis = 0)
X = (X - X_min)/(X_max - X_min)

In [81]:
X

array([[0.0334198 , 0.66666667, 0.5       , ..., 0.5       , 0.        ,
        0.3864598 ],
       [0.03879502, 0.55555556, 0.875     , ..., 0.33333333, 0.33333333,
        0.32440056],
       [0.04650728, 0.66666667, 0.5       , ..., 0.33333333, 0.33333333,
        0.42877292],
       ...,
       [0.03618687, 0.66666667, 1.        , ..., 0.58333333, 0.66666667,
        0.17771509],
       [0.03934189, 0.44444444, 0.625     , ..., 0.25      , 0.        ,
        0.16925247],
       [0.04037019, 0.44444444, 0.625     , ..., 0.33333333, 0.        ,
        0.19464034]])

In [82]:
# X_mean = np.sum(X, axis = 0)/X.shape[0]

In [83]:
# X_stddev = np.sqrt(np.sum((X - X_mean)**2, axis = 0)/X.shape[0])

In [84]:
# X_stddev

In [85]:
# X = (X - X_mean)/X_stddev

In [86]:
Y = np.array(Y)
Y = np.reshape(Y, (Y.shape[0],1))

In [87]:
Y.shape

(1460, 1)

In [88]:
X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size = 0.2, random_state = 42)

In [89]:
X_train[0]

array([0.0331861 , 0.44444444, 0.625     , 0.21505728, 0.33333333,
       0.        , 0.375     , 0.25      , 0.        , 0.20733427])

In [90]:
X_test.shape

(292, 10)

In [180]:
Y_train.shape

(1168, 1)

In [58]:
Y_test.shape

(292,)

In [59]:
X[0].shape

(10,)

In [238]:
NN = NeuralNetwork()
NN.add(DenseLayer (X_train[0].shape[0], 'relu', (X_train[0].shape[0],1)))
NN.add(DenseLayer (20, 'relu'))
NN.add(DenseLayer (12, 'relu'))
NN.add(DenseLayer (1, 'sigmoid'))

In [239]:
NN

In [240]:
len(NN.layers)

4

In [241]:
for epoch in range(200):
    for i in range(X_train.shape[0]):
        NN.fit(X_train[i], Y_train[i])

AttributeError: 'int' object has no attribute 'T'

In [176]:
for i in range(X_test.shape[0]):
    NN.predict(X_test[i],Y_test[i])

pred [[0.05528316]]
pred [[0.99361745]]
pred [[0.09847571]]
pred [[0.87753106]]
pred [[0.9953634]]
pred [[0.00120325]]
pred [[0.96814011]]
pred [[0.83275512]]
pred [[0.00121571]]
pred [[0.07337183]]
pred [[0.69231151]]
pred [[0.00772762]]
pred [[0.87304812]]
pred [[0.8969229]]
pred [[0.86871339]]
pred [[0.02530488]]
pred [[0.93903043]]
pred [[0.02563252]]
pred [[0.13572819]]
pred [[0.88038159]]
pred [[0.38872055]]
pred [[0.96929141]]
pred [[0.70205188]]
pred [[0.01865796]]
pred [[0.88825758]]
pred [[0.61280983]]
pred [[0.79822749]]
pred [[0.00521781]]
pred [[0.87439549]]
pred [[0.94007524]]
pred [[0.0111049]]
pred [[0.99291457]]
pred [[0.79547407]]
pred [[0.00726374]]
pred [[0.99530502]]
pred [[0.1654427]]
pred [[0.05818913]]
pred [[0.95791597]]
pred [[0.99887926]]
pred [[0.00056]]
pred [[0.14999804]]
pred [[0.94654699]]
pred [[0.00964221]]
pred [[0.99971439]]
pred [[0.01691006]]
pred [[0.09805656]]
pred [[0.01467449]]
pred [[0.01692303]]
pred [[0.99961181]]
pred [[0.05203084]]
pred [[

In [177]:
1 - NN.train_accuracy/(X_train.shape[0]*200)

array([0.88471747])

In [178]:
1 - NN.test_accuracy/X_test.shape[0]

array([[0.91438356]])

In [127]:
np.random.seed(42)

In [144]:
NN.train_accuracy

array([[14995]])

In [213]:
model = NeuralNetwork()
model.add_layer(DenseLayer (X_train[0].shape[0], 'relu', (X_train[0].shape[0],X_train.shape[0])))
model.add_layer(DenseLayer (20, 'relu'))
model.add_layer(DenseLayer (12, 'relu'))
model.add_layer(DenseLayer (1, 'sigmoid'))

In [214]:
for epoch in range(1000):
    model.fit(X_train.T, Y_train.T)

In [215]:
1 - model.train_accuracy

array([0.86288312])

In [199]:
model.predict(X_test.T, Y_test.T)

ValueError: operands could not be broadcast together with shapes (20,292) (20,1168) 